In [1]:
!pip install -q tensorflow-recommenders

In [2]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [3]:
rating_df = pd.read_csv('..\Dataset\dataset_dummy.csv')
rating_df.head(100)

,user_id,nama,genre_seni,genre_seni_id,rating
0,1,User_1,Renaissance,9,3
1,1,User_1,Abstract,1,5
2,1,User_1,Impressionism,5,9
3,1,User_1,Dadaism,3,7
4,2,User_2,Fauvism,4,5
...,...,...,...,...,...
95,18,User_18,Dadaism,3,1
96,18,User_18,Renaissance,9,3
97,18,User_18,Nouveau,6,1
98,18,User_18,Surrealism,10,2


In [4]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        552 non-null    int64 
 1   nama           552 non-null    object
 2   genre_seni     552 non-null    object
 3   genre_seni_id  552 non-null    int64 
 4   rating         552 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 21.7+ KB


In [5]:
# Melihat jumlah masing-masing nilai unik dalam kolom "genre_seni"
genre_counts = rating_df['genre_seni'].value_counts()

# Membuat dataframe dari nilai unik dan jumlah kemunculannya
genre_counts_df = pd.DataFrame(genre_counts.items(), columns=[
                               'Genre Seni', 'Jumlah'])

# Mencetak dataframe
print(genre_counts_df)

      Genre Seni  Jumlah
0  Impressionism      61
1        Dadaism      60
2        Fauvism      59
3    Renaissance      57
4         Cubism      56
5        Realism      55
6     Surrealism      52
7        Nouveau      52
8            Pop      51
9       Abstract      49


In [6]:
# Menghitung jumlah unique user_id
jumlah_unique_user = rating_df['user_id'].nunique()

# Mencetak jumlah unique user
print("Jumlah Unique User:", jumlah_unique_user)

Jumlah Unique User: 100


In [7]:
# Menghitung jumlah masing-masing rating dan mengurutkannya secara ascending
jumlah_rating = rating_df['rating'].value_counts().sort_index()

# Mencetak jumlah masing-masing rating yang sudah diurutkan secara ascending
print("Jumlah Masing-masing Rating:")
print(jumlah_rating)

Jumlah Masing-masing Rating:
rating
1     51
2     61
3     48
4     56
5     60
6     50
7     53
8     60
9     53
10    60
Name: count, dtype: int64


In [8]:
genre_seni_df = rating_df[['genre_seni_id', 'genre_seni']]

In [9]:
unique_genre_ids = genre_seni_df['genre_seni_id'].unique()

In [10]:
unique_genres = genre_seni_df['genre_seni'].unique()

In [11]:
genre_df = pd.DataFrame(
    {'genre_seni_id': unique_genre_ids, 'genre_seni': unique_genres})
genre_df = genre_df.sort_values('genre_seni_id')
genre_df.head(10)

,genre_seni_id,genre_seni
1,1,Abstract
7,2,Cubism
3,3,Dadaism
4,4,Fauvism
2,5,Impressionism
9,6,Nouveau
6,7,Pop
5,8,Realism
0,9,Renaissance
8,10,Surrealism


In [12]:
rating_df.head(10)

,user_id,nama,genre_seni,genre_seni_id,rating
0,1,User_1,Renaissance,9,3
1,1,User_1,Abstract,1,5
2,1,User_1,Impressionism,5,9
3,1,User_1,Dadaism,3,7
4,2,User_2,Fauvism,4,5
5,3,User_3,Realism,8,3
6,3,User_3,Pop,7,1
7,3,User_3,Renaissance,9,2
8,3,User_3,Impressionism,5,7
9,3,User_3,Fauvism,4,1


In [13]:
rating_df['user_id'] = rating_df['user_id'].astype(str)
genre_df['genre_seni_id'] = rating_df['genre_seni_id'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(
    dict(rating_df[['user_id', 'genre_seni', 'rating']]))

genres = tf.data.Dataset.from_tensor_slices(
    dict(genre_df[['genre_seni']]))

ratings = ratings.map(lambda x: {
    "genre_seni": x["genre_seni"],
    "user_id": x["user_id"],
    "rating": int(x["rating"])
})

genres = genres.map(lambda x: x["genre_seni"])

In [14]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        552 non-null    object
 1   nama           552 non-null    object
 2   genre_seni     552 non-null    object
 3   genre_seni_id  552 non-null    int64 
 4   rating         552 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 21.7+ KB


In [15]:
genre_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 1 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   genre_seni_id  10 non-null     object
 1   genre_seni     10 non-null     object
dtypes: object(2)
memory usage: 240.0+ bytes


In [16]:
# SEED = 16
# SHUFFLE = 8
# BATCH_SIZE = 100

In [17]:
# Set the seed for reproducibility
tf.random.set_seed(16)
shuffled = ratings.shuffle(8, seed=16, reshuffle_each_iteration=False)

# Calculate the size of the training and testing sets
train_size = int(len(ratings) * 0.8)
test_size = len(ratings) - train_size

# Split the dataset
train = ratings.take(train_size)
test = ratings.skip(train_size).take(test_size)

# Print the sizes of the training and testing sets
print('Training set size:', len(train))
print('Testing set size:', len(test))

Training set size: 441
Testing set size: 111


In [18]:
genre_seni = genres.batch(100)
user_ids = ratings.batch(100).map(lambda x: x["user_id"])

unique_genre_titles = np.unique(np.concatenate(list(genre_seni)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_genre_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 10
Unique users: 100


In [19]:
class GenreModels(tfrs.models.Model):

    def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
        # We take the loss weights in the constructor: this allows us to instantiate
        # several model objects with different loss weights.

        super().__init__()

        embedding_dimension = 64

        # User and movie models.
        self.genre_model: tf.keras.layers.Layer = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_genre_titles, mask_token=None),
            tf.keras.layers.Embedding(
                len(unique_genre_titles) + 1, embedding_dimension)
        ])
        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(
                len(unique_user_ids) + 1, embedding_dimension)
        ])

        # A small model to take in user and movie embeddings and predict ratings.
        # We can make this as complicated as we want as long as we output a scalar
        # as our prediction.
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            # tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

        # The tasks.
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=genres.batch(128).map(self.genre_model)
            )
        )

        # The loss weights.
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])
        # And pick out the movie features and pass them into the movie model.
        genre_embeddings = self.genre_model(features["genre_seni"])

        return (
            user_embeddings,
            genre_embeddings,
            # We apply the multi-layered rating model to a concatentation of
            # user and movie embeddings.
            self.rating_model(
                tf.concat([user_embeddings, genre_embeddings], axis=1)
            ),
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        ratings = features.pop("rating")

        user_embeddings, genre_embeddings, rating_predictions = self(features)

        # We compute the loss for each task.
        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )
        retrieval_loss = self.retrieval_task(user_embeddings, genre_embeddings)

        # And combine them using the loss weights.
        return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

In [36]:
model = GenreModels(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.001))

cached_train = train.shuffle(8).batch(100).cache()
cached_test = test.batch(100).cache()

model.fit(cached_train, epochs=500)

Epoch 1/500
5/5 [==============================] - 1s 65ms/step - root_mean_squared_error: 6.1625 - factorized_top_k/top_1_categorical_accuracy: 0.0295 - factorized_top_k/top_5_categorical_accuracy: 0.4059 - factorized_top_k/top_10_categorical_accuracy: 0.9184 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 395.4654 - regularization_loss: 0.0000e+00 - total_loss: 395.4654
Epoch 2/500
5/5 [==============================] - 0s 66ms/step - root_mean_squared_error: 6.1085 - factorized_top_k/top_1_categorical_accuracy: 0.0317 - factorized_top_k/top_5_categorical_accuracy: 0.4104 - factorized_top_k/top_10_categorical_accuracy: 0.9252 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 394.8111 - regularization_loss: 0.0000e+00 - total_loss: 394.8111
Epoch 3/500
5/5 [==============================] - 0s 65ms/step - root_mean_squared_error: 6.0614 - factoriz

In [37]:
metrics = model.evaluate(cached_test, return_dict=True)

print(
    f"\nRetrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy'] * 100:.3f}%")
print(f"RMSE: {metrics['root_mean_squared_error']:.3f}")

2/2 [==============================] - 1s 79ms/step - root_mean_squared_error: 3.0433 - factorized_top_k/top_1_categorical_accuracy: 0.0360 - factorized_top_k/top_5_categorical_accuracy: 0.4414 - factorized_top_k/top_10_categorical_accuracy: 0.9730 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 182.0071 - regularization_loss: 0.0000e+00 - total_loss: 182.0071

Retrieval top-5 accuracy: 44.144%
RMSE: 3.043


In [38]:
def predict_genres(user, top_n=5):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    # Recommend genres out of the entire genres dataset.
    index.index_from_dataset(
        tf.data.Dataset.zip(
            (genres.batch(100), genres.batch(100).map(model.genre_model)))
    )

    # Ensure k doesn't exceed the number of unique genres
    num_genres = len(unique_genre_titles)
    k = min(top_n, num_genres)

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]), k=k)

    print(f'Top {top_n} recommendations genre for user {user}:\n')
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print(f'{i+1}. {title.decode("utf-8")}')


def predict_rating(user, genre):
    trained_genre_embeddings, trained_user_embeddings, predicted_rating = model({
        "user_id": np.array([str(user)]),
        "genre_title": np.array([genre])
    })
    print(f"Predicted rating for {genre}: {predicted_rating.numpy()[0][0]}")

In [39]:
rating_df[rating_df['user_id'] == '2']

,user_id,nama,genre_seni,genre_seni_id,rating
4,2,User_2,Fauvism,4,5


In [40]:
predict_genres(2, 5)

Top 5 recommendations genre for user 2:

1. Fauvism
2. Impressionism
3. Cubism
4. Surrealism
5. Nouveau


In [41]:
model.save_weights('tfrs.h5')

In [42]:
# Load the model weights
model.load_weights('tfrs.h5')

# Define genres dataset and unique_genre_titles if not already defined
# Example placeholder (you need to replace this with actual data)
unique_genre_titles = np.unique(np.concatenate(list(genre_seni)))

# Create a tf.data.Dataset of genres
genres = tf.data.Dataset.from_tensor_slices(unique_genre_titles)


def predict_genres(user, top_n=5):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    # Recommend genres out of the entire genres dataset.
    index.index_from_dataset(
        tf.data.Dataset.zip(
            (genres.batch(100), genres.batch(100).map(model.genre_model)))
    )

    # Ensure k doesn't exceed the number of unique genres
    num_genres = len(unique_genre_titles)
    k = min(top_n, num_genres)

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]), k=k)

    print(f'Top {top_n} recommendations genre for user {user}:\n')
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print(f'{i+1}. {title.decode("utf-8")}')


# Example usage:
# Replace 'example_user_id' with the actual user ID you want to predict for.
predict_genres('2', top_n=5)

Top 5 recommendations genre for user 2:

1. Fauvism
2. Impressionism
3. Cubism
4. Surrealism
5. Nouveau


In [43]:
# import numpy as np
# import tensorflow as tf
# import tensorflow_recommenders as tfrs

# # Load the model weights
# model.load_weights('tfrs.h5')

# # Define genres dataset and unique_genre_titles if not already defined
# # Example placeholder (replace this with actual data)
# unique_genre_titles = np.unique(np.concatenate(list(genre_seni)))

# # Create a tf.data.Dataset of genres
# genres = tf.data.Dataset.from_tensor_slices(unique_genre_titles)


# def predict_genres(user, top_n=5):
#     # Create a model that takes in raw query features, and
#     index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

#     # Recommend genres out of the entire genres dataset.
#     index.index_from_dataset(
#         tf.data.Dataset.zip(
#             (genres.batch(100), genres.batch(100).map(model.genre_model)))
#     )

#     # Ensure k doesn't exceed the number of unique genres
#     num_genres = len(unique_genre_titles)
#     k = min(top_n, num_genres)

#     # Get recommendations.
#     _, titles = index(tf.constant([str(user)]), k=k)

#     print(f'Top {top_n} recommended genres for user {user}:\n')
#     for i, title in enumerate(titles[0, :top_n].numpy()):
#         print(f'{i+1}. {title.decode("utf-8")}')


# # Interactive loop to get user input
# while True:
#     user_id = input("Enter user ID (or 'exit' to quit): ")
#     if user_id.lower() == 'exit':
#         break
#     top_n = input("Enter the number of top genres to recommend: ")
#     try:
#         top_n = int(top_n)
#     except ValueError:
#         print("Invalid input for number of genres. Please enter an integer.")
#         continue

#     predict_genres(user_id, top_n)